In [3]:
from AnalyzerCalendar import AnalyzerCalendar as Analyzer
import pandas as pd
from datetime import datetime
import numpy as np
analyzerTEST = Analyzer("TEST/AnalyzerCalendar")
analyzer = Analyzer()

In [4]:
destinations = analyzer.find_trips_between_locations(1.0, 1.0, 2.0, 2.0, datetime(2024, 7, 1), datetime(2024, 8, 1), pd.Timedelta(hours=6))
print(destinations)

Empty DataFrame
Columns: []
Index: []


In [66]:
def is_service_in_dates(services: pd.Series) -> bool:
    if services.days_ok >= pd.Timedelta(days=6):
        return True
    day_offset = 0
    while day_offset <= services.days_ok.days:
        current_day_index = (services.days_ok_start + pd.Timedelta(days=day_offset)).weekday()
        if services.iloc[analyzerTEST.monday_integer_index + current_day_index] == 1:
            return True
        day_offset += 1
    return False

start_date = pd.to_datetime(datetime(2024, 6, 1))
end_date = pd.to_datetime(datetime(2024, 8, 1))
trip_ids: pd.Series = analyzerTEST.get_trips_nearby_location(1.0, 1.0)
#print(trip_ids)
relevant_trips: pd.DataFrame = analyzerTEST.trips[analyzerTEST.trips["trip_id"].isin(trip_ids)]
#print(relevant_trips)
relevant_services: pd.DataFrame = analyzerTEST.calendar[analyzerTEST.calendar["service_id"].isin(relevant_trips["service_id"])]
#print(relevant_services)
services_within_period: pd.DataFrame = relevant_services[
    ((relevant_services["start_date"] >= start_date) & (relevant_services["start_date"] <= end_date))
    | ((relevant_services["end_date"] >= start_date) & (relevant_services["end_date"] <= end_date))
]
#print(services_within_period)
#if services_within_period.empty:
    #return pd.Series()
services_within_period = services_within_period.assign(
    days_ok_start=np.vectorize(lambda x: start_date if (start_date >= x) else x)(
        services_within_period["start_date"]
    ),
    days_ok_end=np.vectorize(lambda x: end_date if (end_date <= x) else x)(services_within_period["end_date"]),
)
services_within_period["days_ok"] = services_within_period["days_ok_end"] - services_within_period["days_ok_start"]
# Optimisation de la ligne suivante à faire
services_in_dates = services_within_period[services_within_period.apply(is_service_in_dates, axis=1)]
services_in_dates.drop_duplicates(subset="service_id")
trips_dans_periode: pd.DataFrame = relevant_trips[relevant_trips["service_id"].isin(services_in_dates["service_id"])]
print(trips_dans_periode)
#print(trips_dans_periode["trip_id"])

   service_id  trip_id  route_id
0           1  TRIP001  ROUTE001


In [31]:

appearance_count = analyzerTEST.stop_times.groupby("stop_id").count()["trip_id"]
appearance_stop_id = pd.merge(appearance_count, analyzerTEST.stops, on="stop_id")
appeareance_stop_area = appearance_stop_id.groupby("parent_station").sum()["trip_id"]
df_stop_area = analyzerTEST.stops_area[["stop_id", "stop_name", "stop_lat", "stop_lon"]].copy()
df_stop_area.set_index("stop_id", inplace=True)
df_stop_area = df_stop_area.assign(number_of_appearance=appeareance_stop_area)
analyzerTEST.city_list = df_stop_area
df_stop_area

        stop_id    stop_name  stop_lat  stop_lon parent_station
0         Laval        Laval    42.000     2.000            NaN
2         Paris        Paris    45.000     5.000            NaN
4        Paris2       Paris2    45.001     5.001            NaN
6        Rennes       Rennes    40.000     0.000            NaN
8        Nantes       Nantes    45.000     0.000            NaN
10      Nantes2      Nantes2    45.001     0.001            NaN
12    Marseille    Marseille    50.000    10.000            NaN
14    Perpignan    Perpignan    48.000     5.000            NaN
16  Carcassonne  Carcassonne   100.000    15.000            NaN
18       Vannes       Vannes   -50.000     5.000            NaN
20   Strasbourg   Strasbourg    90.000     9.000            NaN


,stop_name,stop_lat,stop_lon,number_of_appearance
stop_id,,,,
Laval,Laval,42.000,2.000,4
Paris,Paris,45.000,5.000,2
Paris2,Paris2,45.001,5.001,2
Rennes,Rennes,40.000,0.000,2
Nantes,Nantes,45.000,0.000,2
Nantes2,Nantes2,45.001,0.001,1
Marseille,Marseille,50.000,10.000,1
Perpignan,Perpignan,48.000,5.000,3
Carcassonne,Carcassonne,100.000,15.000,2
